In [1]:
!git clone https://github.com/AntonioTepsich/Convolutional-KANs.git



Cloning into 'Convolutional-KANs'...
remote: Enumerating objects: 1528, done.
remote: Counting objects: 100% (374/374), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 1528 (delta 248), reused 345 (delta 221), pack-reused 1154
Receiving objects: 100% (1528/1528), 33.37 MiB | 28.06 MiB/s, done.
Resolving deltas: 100% (722/722), done.


In [2]:
!mv Convolutional-KANs ConvolutionalKANs
import sys
sys.path.append('/content/ConvolutionalKANs/kan_convolutional')
sys.path.append('/content/ConvolutionalKANs')

In [3]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.1

  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached torchvision-0.15.2-cp310-cp310-manylinux1_x86_64.whl (6.0 MB)
  Using cached torchaudio-2.0.1-cp310-cp310-manylinux1_x86_64.whl (4.4 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (

In [4]:
!sed -i '/torch==2.3.0+cu118/d' ConvolutionalKANs/requirements.txt
!pip install -r ConvolutionalKANs/requirements.txt

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, Subset
from ConvolutionalKANs.architectures_28x28.CKAN_BN import CKAN_BN
from ConvolutionalKANs.architectures_28x28.SimpleModels import SimpleCNN
from ConvolutionalKANs.architectures_28x28.ConvNet import ConvNet
from ConvolutionalKANs.architectures_28x28.KANConvs_MLP import KANC_MLP
from ConvolutionalKANs.architectures_28x28.KKAN import KKAN_Convolutional_Network
from ConvolutionalKANs.architectures_28x28.conv_and_kan import NormalConvsKAN

In [20]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Training set
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
sorted_indices = sorted(range(len(train_dataset)), key=lambda idx: train_dataset.targets[idx].item())
subset_indices = sorted_indices[:len(train_dataset)//20]
train_subset = Subset(train_dataset, subset_indices)

# Test set
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_subset_indices = list(range(len(test_dataset)//10))
test_subset = Subset(test_dataset, test_subset_indices)

train_loader = DataLoader(train_subset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)


print(f'Lunghezza del dataset di addestramento (subset): {len(train_subset)}')
print(f'Lunghezza del dataset di test (subset): {len(test_subset)}')


Lunghezza del dataset di addestramento (subset): 3000
Lunghezza del dataset di test (subset): 1000


In [21]:
def train(model, train_loader, optimizer, epoch, criterion):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_loss = train_loss / (batch_idx + 1)
    return avg_loss

def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = torch.max(output.data, 1)
            correct += (target == predicted).sum().item()
    test_loss /= len(test_loader)
    accuracy = correct / len(test_loader.dataset)
    return test_loss, accuracy


In [22]:
# Ciao :(
ckan_model = CKAN_BN()
cnn_model = SimpleCNN()


criterion = nn.CrossEntropyLoss()
ckan_optimizer = optim.Adam(ckan_model.parameters(), lr=0.001)
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# CKAN
print("Training CKAN Model")
for epoch in range(1, 6):
    train_loss = train(ckan_model, train_loader, ckan_optimizer, epoch, criterion)
    test_loss, accuracy = test(ckan_model, test_loader, criterion)
    print(f'End of Epoch {epoch}: Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2%}')

# SimpleCNN
print("Training SimpleCNN Model")
for epoch in range(1, 6):
    train_loss = train(cnn_model, train_loader, cnn_optimizer, epoch, criterion)
    test_loss, accuracy = test(cnn_model, test_loader, criterion)
    print(f'End of Epoch {epoch}: Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2%}')

Training CKAN Model
End of Epoch 1: Train Loss: 0.050292, Test Loss: 14.0552, Accuracy: 8.50%
End of Epoch 2: Train Loss: 0.000000, Test Loss: 21.4268, Accuracy: 8.50%
End of Epoch 3: Train Loss: 0.000000, Test Loss: 21.5012, Accuracy: 8.50%
End of Epoch 4: Train Loss: 0.000000, Test Loss: 21.6275, Accuracy: 8.50%
End of Epoch 5: Train Loss: 0.000000, Test Loss: 21.5829, Accuracy: 8.50%
Training SimpleCNN Model
End of Epoch 1: Train Loss: 0.861568, Test Loss: 7.1968, Accuracy: 8.50%
End of Epoch 2: Train Loss: 0.000325, Test Loss: 9.3860, Accuracy: 8.50%
End of Epoch 3: Train Loss: 0.000098, Test Loss: 9.7662, Accuracy: 8.50%
End of Epoch 4: Train Loss: 0.000061, Test Loss: 10.2425, Accuracy: 8.50%
End of Epoch 5: Train Loss: 0.000034, Test Loss: 10.8066, Accuracy: 8.50%
